In [1]:
import torch
from torchvision import datasets
from torchvision.transforms import ToTensor, Normalize

In [2]:
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from IPython.core.debugger import set_trace

In [3]:
# setting device and data types
dtype = torch.float32 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
import torch.nn as nn

In [5]:
'''
Input channels will be:
    - x, x coordinate position.
    - y, y coordinate position.
    - u, velocity in the x direction.
    - v, velocity in the y direction.
    - p, pressure field.

Output channels, will have three:
    - u, velocity in the x direction.
    - v, velocity in the y direction.
    - p, pressure field.

Can I, in a way, add equivariant constraints? Enfore that the model is equivariant.
Add padding to allow for the boundary conditions to exist. Also allows for data augmentation, 
which should prevent overfitting on learning the boundary conditions.

Model trained as an autoencoder, with a PDE loss. We expect that the PDE loss contradicts the AE.
Maybe, could start with the AE to be noearly convergent, and then add the PDE. PDE alone might fail at
performing the auto-encoding overall.
'''

'\nInput channels will be:\n    - x, x coordinate position.\n    - y, y coordinate position.\n    - u, velocity in the x direction.\n    - v, velocity in the y direction.\n    - p, pressure field.\n\nOutput channels, will have three:\n    - u, velocity in the x direction.\n    - v, velocity in the y direction.\n    - p, pressure field.\n\nCan I, in a way, add equivariant constraints? Enfore that the model is equivariant.\nAdd padding to allow for the boundary conditions to exist. Also allows for data augmentation, \nwhich should prevent overfitting on learning the boundary conditions.\n\nModel trained as an autoencoder, with a PDE loss. We expect that the PDE loss contradicts the AE.\nMaybe, could start with the AE to be noearly convergent, and then add the PDE. PDE alone might fail at\nperforming the auto-encoding overall.\n'

In [6]:
class Interpolate(nn.Module):
    def __init__(self, size, mode):
        super(Interpolate, self).__init__()
        self.interp = nn.functional.interpolate
        self.size = size
        self.mode = mode
        
    def forward(self, x):
        x = self.interp(x, size=self.size, mode=self.mode, align_corners=False)
        return x

In [7]:
from torch.nn.modules.activation import ReLU
class ConvAE(nn.Module):
    def __init__(self, input_size):
        super(ConvAE, self).__init__()

        # encoder
        self.enc1 = nn.Sequential(
            nn.Conv2d(in_channels=5, out_channels=16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            Interpolate(size=input_size//2, mode='bilinear'),
        )

        self.enc2 = nn.Sequential(
          nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1),
          nn.ReLU(),
          Interpolate(size=input_size//8, mode='bilinear'),
        )
        
        self.enc3 = nn.Sequential(
          nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
          nn.ReLU(),
          Interpolate(size=input_size//16, mode='bilinear'),
        )

        # decoder 
        self.dec1 = nn.Sequential(
          nn.Conv2d(in_channels=64, out_channels=32, kernel_size=3, stride=1, padding=1),
          nn.ReLU(),
          Interpolate(size=input_size//8, mode='bilinear'),
        )
        self.dec2 = nn.Sequential(
          nn.Conv2d(in_channels=32, out_channels=16, kernel_size=3, stride=1, padding=1),
          nn.ReLU(),
          Interpolate(size=input_size//2, mode='bilinear'),
        )
        self.dec3 = nn.Sequential(
          nn.Conv2d(in_channels=16, out_channels=5, kernel_size=3, stride=1, padding=1),
          nn.ReLU(),
          Interpolate(size=input_size, mode='bilinear'),
        )
        self.dec4 = nn.Sequential(
          nn.Conv2d(in_channels=5, out_channels=3, kernel_size=3, stride=1, padding=1),
        )
    
    def compute_next_size(self, dimension, kernel, padding=0, stride=1):
        return int((dimension + 2*padding - kernel) / stride + 1)

    def forward(self, x):
        # encoding
        x = self.enc1(x)
        x = self.enc2(x)
        x = self.enc3(x)
        
        # decoding
        x = self.dec1(x)
        x = self.dec2(x)
        x = self.dec3(x)
        reconstruction = self.dec4(x)
        return reconstruction

In [8]:
input_size=320
model = ConvAE(input_size=input_size)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [9]:
# loss function? will have to think about it :)

In [10]:
x = torch.randn((10, input_size, input_size, 1), requires_grad=True)
y = torch.randn((10, input_size, input_size, 1), requires_grad=True)
u = torch.randn((10, input_size, input_size, 1), requires_grad=True)
v = torch.randn((10, input_size, input_size, 1), requires_grad=True)
p = torch.randn((10, input_size, input_size, 1), requires_grad=True)

In [11]:
flow_input = torch.cat([x, y, u, v, p], axis=-1)
flow_input = flow_input.permute((0, 3, 1, 2))
flow_input.shape

torch.Size([10, 5, 320, 320])

In [12]:
optimizer.zero_grad()
flow_output = model(flow_input)
flow_output.shape

torch.Size([10, 3, 320, 320])

In [13]:
# upred = (x**2*y+y**2)*u
# vpred = (y*x)**2*v
# ppred = x*v + y*u
# flow_output = torch.cat((upred, vpred, ppred), axis=-1)
# flow_output.shape

In [14]:
upred = flow_output[:, 0]
vpred = flow_output[:, 1]
ppred = flow_output[:, 2]

In [45]:
from loss_functions import PDELoss
pde_loss_function = PDELoss()

In [46]:
pde_loss = pde_loss_function.compute_loss(flow_input[:, 0], flow_input[:, 1], flow_output[:, 0], flow_output[:, 1], flow_output[:, 2])

IndexError: index 3 is out of bounds for dimension 1 with size 3

In [44]:
dudx, dudy = torch.autograd.grad(upred, [x, y], grad_outputs=torch.ones(upred.shape), retain_graph=True, create_graph=True)
dvdx, dvdy = torch.autograd.grad(vpred, [x, y], grad_outputs=torch.ones(vpred.shape), retain_graph=True, create_graph=True)
dpdx, dpdy = torch.autograd.grad(ppred, [x, y], grad_outputs=torch.ones(ppred.shape), retain_graph=True)

In [17]:
dudx.shape

torch.Size([10, 320, 320, 1])

In [18]:
d2udx2 = torch.autograd.grad(dudx.squeeze(), x, grad_outputs=torch.ones(dudx.squeeze().shape), retain_graph=True)[0]
d2udy2 = torch.autograd.grad(dudy.squeeze(), y, grad_outputs=torch.ones(dudy.squeeze().shape), retain_graph=True)[0]
d2vdx2 = torch.autograd.grad(dvdx.squeeze(), x, grad_outputs=torch.ones(dvdx.squeeze().shape), retain_graph=True)[0]
d2vdy2 = torch.autograd.grad(dvdy.squeeze(), y, grad_outputs=torch.ones(dvdy.squeeze().shape), retain_graph=True)[0]

In [19]:
# pde_loss, each navier stokes term difference squared!
ae_loss_function = nn.MSELoss(reduction='mean')
ae_loss = ae_loss_function(flow_input[:, 2:], flow_output)

In [20]:
''' assume steady state Navier-Stokes...? No, must have a time component somewhere. I remeber thinking about this.
Where am I suppose to put this time component? add it in the latent space? with the Reynold's number?
Or with the density, the viscosity?
Well, if you non-dmiensionalise, you just need the Re, that's the point, but you also need to non-dimensinoalise
the distance, etc... 
'''

" assume steady state Navier-Stokes...? No, must have a time component somewhere. I remeber thinking about this.\nWhere am I suppose to put this time component? add it in the latent space? with the Reynold's number?\nOr with the density, the viscosity?\nWell, if you non-dmiensionalise, you just need the Re, that's the point, but you also need to non-dimensinoalise\nthe distance, etc... \n"

In [21]:
rho = 1
mu = 1
beta = 0.5

In [22]:
upred.shape, d2udx2.shape

(torch.Size([10, 320, 320]), torch.Size([10, 320, 320, 1]))

In [23]:
continuity_error = torch.sum((dudx.squeeze() + dvdy.squeeze())**2)
xmomentum_error = torch.sum((rho*(upred*dudx.squeeze() + vpred*dudy.squeeze()) + dpdx.squeeze() - mu*(d2udx2.squeeze() + d2udy2.squeeze()))**2)
ymomentum_error = torch.sum((rho*(upred*dvdx.squeeze() + vpred*dvdy.squeeze()) + dpdy.squeeze() - mu*(d2vdx2.squeeze() + d2vdy2.squeeze()))**2)
loss = beta*ae_loss + (1-beta)*(continuity_error + xmomentum_error + ymomentum_error)

In [37]:
loss.cpu().detach().numpy()

array(1.9337444, dtype=float32)